# Retail Insight Pro: Data-Driven Analysis for Optimizing Retail Operations

---------------------------------------------------------------------------------------------------------------------------

### 1. Import Libraries

Import essential Python libraries like pandas, numpy, matplotlib.pyplot, and seaborn for data manipulation, numerical operations, and visualization.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

--------------------------------------------------------------------------------------------------------------------------

## Download Retail_data and Modify Changes Accordingly

### 1. modify the dataset by adding 2025 data 

In [23]:
df=pd.read_csv("D:/RAHUL-SIR-MCA-PROJECT/Raw_Data/Kaggle_Retail_data.csv")

In [24]:
import numpy as np

# Define growth factor for 2025 sales (e.g., 40% increase over 2024 sales count)
growth_factor = 1.4
sales_2025_count = int(len(df[df['Year'] == 2024.0]) * growth_factor)

# Sample records from 2024 to create synthetic 2025 data
df_2025 = df[df['Year'] == 2024.0].sample(n=sales_2025_count, replace=True).copy()

# Update the year to 2025
df_2025['Year'] = 2025

# Increase total sales amount for 2025 by 15-25%
df_2025['Total_Amount'] *= np.random.uniform(1.15, 1.25, size=sales_2025_count)

# Combine with original dataset
df_updated = pd.concat([df, df_2025], ignore_index=True)

# Verify the new dataset distribution
df_updated['Year'].value_counts()


2023.0    251852
2025.0     69731
2024.0     49808
Name: Year, dtype: int64

### 2. Save this file

In [25]:
# Save the updated dataset
updated_file_path = "D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/Retail_data.csv"
df_updated.to_csv(updated_file_path, index=False)

# Return the file path for reference
updated_file_path


'D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/Retail_data.csv'

### 3. Date column to be updated correctly along with the Year column

In [26]:
import pandas as pd

# Ensure Date column is in datetime format
df_2025['Date'] = pd.to_datetime(df_2025['Date'], errors='coerce')

# Function to replace year while handling leap year issues
def update_year(date):
    if pd.notnull(date):
        try:
            return date.replace(year=2025)  # Change year to 2025
        except ValueError:  
            return date.replace(year=2025, day=28)  # Adjust Feb 29 to Feb 28
    return None

# Apply the function to update dates
df_2025['Date'] = df_2025['Date'].apply(update_year)


In [16]:
from datetime import datetime, timedelta

# Convert 'Date' column to datetime format (handling missing values)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Ensure Date column is in datetime format
df_2025['Date'] = pd.to_datetime(df_2025['Date'], errors='coerce')

# Generate new 2025 dates while keeping the same month-day structure
df_2025['Date'] = df_2025['Date'].apply(lambda x: x.replace(year=2025) if pd.notnull(x) else None)

# Merge the updated dataset
df_updated = pd.concat([df, df_2025], ignore_index=True)

# Verify changes in the Date column for 2025
df_updated[df_updated['Year'] == 2025]['Date'].describe()


C:\Users\PRADEEP TIPPE\AppData\Local\Temp\ipykernel_10928\4227341894.py:16: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_updated[df_updated['Year'] == 2025]['Date'].describe()


count                   69644
unique                     59
top       2025-02-28 00:00:00
freq                     2309
first     2025-01-01 00:00:00
last      2025-02-28 00:00:00
Name: Date, dtype: object

In [27]:
# Save the updated dataset
df_updated.to_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/Retail_data.csv", index=False)


### 4.  Generating Ship Date & Delivery Date

In [19]:
print(df_updated.columns)

Index(['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Address',
       'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income',
       'Customer_Segment', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases',
       'Amount', 'Total_Amount', 'Product_Category', 'Product_Brand',
       'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method',
       'Order_Status', 'Ratings', 'products'],
      dtype='object')


In [28]:
import numpy as np

data=pd.read_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/Retail_data.csv")

# Ensure Order Date is in datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Function to generate Ship Date & Delivery Date
def generate_dates(row):
    if pd.notnull(row['Date']) and pd.notnull(row['Customer_Segment']):
        if row['Customer_Segment'] == 'Regular':
            ship_days = 4
            delivery_days = np.random.choice([2, 3])  # Randomly pick 2 or 3 days
        elif row['Customer_Segment'] == 'Premium':
            ship_days = 3
            delivery_days = np.random.choice([1, 2])  # Randomly pick 1 or 2 days
        elif row['Customer_Segment'] == 'New':
            ship_days = 4
            delivery_days = np.random.choice([3, 4])  # Randomly pick 3 or 4 days
        else:
            return None, None  # If segment is unknown

        # Compute Ship Date and Delivery Date
        ship_date = row['Date'] + pd.Timedelta(days=ship_days)
        delivery_date = ship_date + pd.Timedelta(days=delivery_days)
        return ship_date, delivery_date
    return None, None

# Apply function to update dataset
data[['Ship Date', 'Delivery Date']] = data.apply(generate_dates, axis=1, result_type='expand')


### 5.  Generate Product IDs

In [29]:
import pandas as pd
import random

# Generate a unique Product ID
def generate_product_id():
    return f"{random.randint(10000, 99999)}"

# Ensure 'Product Name' column exists
if 'products' in data.columns:
    data['Product ID'] = data['products'].apply(lambda x: generate_product_id())

# Check the first few rows
print(data[['products', 'Product ID']].head())


            products Product ID
0     Cycling shorts      69417
1         Lenovo Tab      24176
2   Sports equipment      48029
3      Utility knife      89705
4  Chocolate cookies      88714


In [30]:
data.to_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/Retail_data.csv", index=False)

---------------------------------------------------------------------------------------------------------------------------

## Preprocess data and save into Bronze Layer

### 1. generate three separate CSV files 

* customer.csv
* order.csv
* product.csv

In [31]:
print(data.columns)

Index(['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Address',
       'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income',
       'Customer_Segment', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases',
       'Amount', 'Total_Amount', 'Product_Category', 'Product_Brand',
       'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method',
       'Order_Status', 'Ratings', 'products', 'Ship Date', 'Delivery Date',
       'Product ID'],
      dtype='object')


In [35]:
import pandas as pd


# Separate columns into different DataFrames as per requirements

# Customer Data
customer_columns = [
    "Customer_ID", "Name", "Email", "Phone", "Address", "City", "State", "Zipcode", 
    "Country", "Age", "Gender", "Income", "Customer_Segment", "Feedback", "Ratings", 
    "Transaction_ID", "Product ID"
]
customer_df = data[customer_columns].drop_duplicates()

# Order Data
order_columns = [
    "Transaction_ID", "Date", "Ship Date", "Delivery Date", "Year", "Month", "Time", 
    "Total_Purchases", "Amount", "Total_Amount", "Shipping_Method", "Payment_Method", 
    "Order_Status", "Product ID", "Customer_ID"
]
order_df = data[order_columns].drop_duplicates()

# Product Data
product_columns = [
    "Product ID", "products", "Product_Category", "Product_Brand", "Product_Type", 
    "Customer_ID", "Transaction_ID"
]
product_df = data[product_columns].drop_duplicates()

# Save files as CSV
customer_csv_path = "D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/customer.csv"
order_csv_path = "D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/order.csv"
product_csv_path = "D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/product.csv"

customer_df.to_csv(customer_csv_path, index=False)
order_df.to_csv(order_csv_path, index=False)
product_df.to_csv(product_csv_path, index=False)

customer_csv_path, order_csv_path, product_csv_path


('D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/customer.csv',
 'D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/order.csv',
 'D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/product.csv')

## 2. Exploratory Data Analysis

Key Purposes of EDA:

* Loading Data
* Basic Data Information
* Handling Missing Data
* check Duplicates
* Data Preprocessing

## 2.1 Loading Data

In [36]:
B_retail_data=pd.read_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/Retail_data.csv")
B_retail_data.head(5)

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products,Ship Date,Delivery Date,Product ID
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts,2023-09-22,2023-09-24,69417
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab,2024-01-03,2024-01-04,24176
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment,2023-04-30,2023-05-03,48029
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife,2023-05-11,2023-05-13,89705
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies,2024-01-13,2024-01-14,88714


In [37]:
B_customer=pd.read_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/customer.csv")
B_customer.head(5)

,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Feedback,Ratings,Transaction_ID,Product ID
0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,Excellent,5.0,8691788.0,69417
1,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium,Excellent,4.0,2174773.0,24176
2,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular,Average,2.0,6679610.0,48029
3,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,56.0,Male,High,Premium,Excellent,4.0,7232460.0,89705
4,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,22.0,Male,Low,Premium,Bad,1.0,4983775.0,88714


In [38]:
B_order=pd.read_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/order.csv")
B_order.head(5)

,Transaction_ID,Date,Ship Date,Delivery Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Shipping_Method,Payment_Method,Order_Status,Product ID,Customer_ID
0,8691788.0,2023-09-18,2023-09-22,2023-09-24,2023.0,September,22:03:55,3.0,108.028757,324.086270,Same-Day,Debit Card,Shipped,69417,37249.0
1,2174773.0,2023-12-31,2024-01-03,2024-01-04,2023.0,December,8:42:04,2.0,403.353907,806.707815,Standard,Credit Card,Processing,24176,69749.0
2,6679610.0,2023-04-26,2023-04-30,2023-05-03,2023.0,April,4:06:29,3.0,354.477600,1063.432799,Same-Day,Credit Card,Processing,48029,30192.0
3,7232460.0,2023-05-08,2023-05-11,2023-05-13,2023.0,May,14:55:17,7.0,352.407717,2466.854021,Standard,PayPal,Processing,89705,62101.0
4,4983775.0,2024-01-10,2024-01-13,2024-01-14,2024.0,January,16:54:07,2.0,124.276524,248.553049,Standard,Cash,Shipped,88714,27901.0


In [39]:
B_product=pd.read_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Bronz_Data/product.csv")
B_product.head(5)

,Product ID,products,Product_Category,Product_Brand,Product_Type,Customer_ID,Transaction_ID
0,69417,Cycling shorts,Clothing,Nike,Shorts,37249.0,8691788.0
1,24176,Lenovo Tab,Electronics,Samsung,Tablet,69749.0,2174773.0
2,48029,Sports equipment,Books,Penguin Books,Children's,30192.0,6679610.0
3,89705,Utility knife,Home Decor,Home Depot,Tools,62101.0,7232460.0
4,88714,Chocolate cookies,Grocery,Nestle,Chocolate,27901.0,4983775.0


## 2.2 Basic Data Infomation

In [43]:
print("Retail data Infomation")
print("\n")
B_retail_data.info()
print("\n")
print("Customer Infomation")
print("\n")
B_customer.info()
print("\n")
print("Order Information")
print("\n")
B_order.info()
print("\n")
print("Product Information")
print("\n")
B_product.info()

Retail data Infomation


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371741 entries, 0 to 371740
Data columns (total 33 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    371316 non-null  float64
 1   Customer_ID       371359 non-null  float64
 2   Name              371271 non-null  object 
 3   Email             371326 non-null  object 
 4   Phone             371294 non-null  float64
 5   Address           371364 non-null  object 
 6   City              371444 non-null  object 
 7   State             371395 non-null  object 
 8   Zipcode           371313 non-null  float64
 9   Country           371410 non-null  object 
 10  Age               371531 non-null  float64
 11  Gender            371371 non-null  object 
 12  Income            371374 non-null  object 
 13  Customer_Segment  371468 non-null  object 
 14  Date              371294 non-null  object 
 15  Year              371391 non-null  float64


## 2.3 Handling Missing Data

### B_Retail_data

In [45]:
B_retail_data.isnull().sum()

Transaction_ID      425
Customer_ID         382
Name                470
Email               415
Phone               447
Address             377
City                297
State               346
Zipcode             428
Country             331
Age                 210
Gender              370
Income              367
Customer_Segment    273
Date                447
Year                350
Month               344
Time                441
Total_Purchases     434
Amount              426
Total_Amount        425
Product_Category    338
Product_Brand       338
Product_Type          0
Feedback            231
Shipping_Method     410
Payment_Method      371
Order_Status        286
Ratings             231
products              0
Ship Date           720
Delivery Date       720
Product ID            0
dtype: int64

In [46]:
B_retail_data=B_retail_data.dropna()

In [47]:
B_retail_data.isnull().sum()

Transaction_ID      0
Customer_ID         0
Name                0
Email               0
Phone               0
Address             0
City                0
State               0
Zipcode             0
Country             0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Date                0
Year                0
Month               0
Time                0
Total_Purchases     0
Amount              0
Total_Amount        0
Product_Category    0
Product_Brand       0
Product_Type        0
Feedback            0
Shipping_Method     0
Payment_Method      0
Order_Status        0
Ratings             0
products            0
Ship Date           0
Delivery Date       0
Product ID          0
dtype: int64

### B_Customer 

In [48]:
B_customer.isnull().sum()

Customer_ID         382
Name                470
Email               415
Phone               447
Address             377
City                297
State               346
Zipcode             428
Country             331
Age                 210
Gender              370
Income              367
Customer_Segment    273
Feedback            231
Ratings             231
Transaction_ID      425
Product ID            0
dtype: int64

In [49]:
B_customer=B_customer.dropna()

In [50]:
B_customer.isnull().sum()

Customer_ID         0
Name                0
Email               0
Phone               0
Address             0
City                0
State               0
Zipcode             0
Country             0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Feedback            0
Ratings             0
Transaction_ID      0
Product ID          0
dtype: int64

### B_Order

In [51]:
B_order.isnull().sum()

Transaction_ID     425
Date               447
Ship Date          720
Delivery Date      720
Year               350
Month              344
Time               441
Total_Purchases    434
Amount             426
Total_Amount       425
Shipping_Method    410
Payment_Method     371
Order_Status       286
Product ID           0
Customer_ID        382
dtype: int64

In [52]:
B_order=B_order.dropna()

In [53]:
B_order.isnull().sum()

Transaction_ID     0
Date               0
Ship Date          0
Delivery Date      0
Year               0
Month              0
Time               0
Total_Purchases    0
Amount             0
Total_Amount       0
Shipping_Method    0
Payment_Method     0
Order_Status       0
Product ID         0
Customer_ID        0
dtype: int64

### B_product

In [54]:
B_product.isnull().sum()

Product ID            0
products              0
Product_Category    338
Product_Brand       338
Product_Type          0
Customer_ID         382
Transaction_ID      425
dtype: int64

In [55]:
B_product=B_product.dropna()

In [56]:
B_product.isnull().sum()

Product ID          0
products            0
Product_Category    0
Product_Brand       0
Product_Type        0
Customer_ID         0
Transaction_ID      0
dtype: int64

## 2.4 Check Duplicates

In [58]:
B_retail_data.duplicated().sum()

0

In [59]:
B_customer.duplicated().sum()

0

In [60]:
B_order.duplicated().sum()

0

In [61]:
B_product.duplicated().sum()

0

## 2.5 Data Preprocessing

### B_Retail_data - Change Datatype

In [66]:
cols = ['Transaction_ID', 'Customer_ID', 'Zipcode','Age','Year','Product ID']  # Specify column names
B_retail_data[cols] = B_retail_data[cols].apply(lambda x: x.astype(int))

In [67]:
B_retail_data[['Transaction_ID', 'Customer_ID', 'Zipcode','Age','Year','Product ID']].dtypes

Transaction_ID    int32
Customer_ID       int32
Zipcode           int32
Age               int32
Year              int32
Product ID        int32
dtype: object

In [68]:
B_retail_data.head(3)

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products,Ship Date,Delivery Date,Product ID
0,8691788,37249,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,...,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts,2023-09-22,2023-09-24,69417
1,2174773,69749,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071,UK,...,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab,2024-01-03,2024-01-04,24176
2,6679610,30192,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929,Australia,...,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment,2023-04-30,2023-05-03,48029


### B_customer - Generate Birth Year column based on the Age column 

In [57]:
from datetime import datetime

# Get the current year
current_year = datetime.now().year

# Create the Birth Year column
B_customer['Birth_Year'] = current_year - B_customer['Age']

# Display the first few rows to verify
print(B_customer[['Age', 'Birth_Year']].head())


    Age  Birth_Year
0  21.0      2004.0
1  19.0      2006.0
2  48.0      1977.0
3  56.0      1969.0
4  22.0      2003.0


### Change Datatype

In [69]:
B_customer.head(3)

,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Feedback,Ratings,Transaction_ID,Product ID,Birth_Year
0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,Excellent,5.0,8691788.0,69417,2004.0
1,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium,Excellent,4.0,2174773.0,24176,2006.0
2,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular,Average,2.0,6679610.0,48029,1977.0


In [70]:
cols = ['Transaction_ID', 'Customer_ID', 'Zipcode','Age','Birth_Year','Product ID']  # Specify column names
B_customer[cols] = B_customer[cols].apply(lambda x: x.astype(int))

In [71]:
B_customer[['Transaction_ID', 'Customer_ID', 'Zipcode','Age','Birth_Year','Product ID']].dtypes

Transaction_ID    int32
Customer_ID       int32
Zipcode           int32
Age               int32
Birth_Year        int32
Product ID        int32
dtype: object

In [72]:
B_customer.head(3)

,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Feedback,Ratings,Transaction_ID,Product ID,Birth_Year
0,37249,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,21,Male,Low,Regular,Excellent,5.0,8691788,69417,2004
1,69749,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071,UK,19,Female,Low,Premium,Excellent,4.0,2174773,24176,2006
2,30192,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929,Australia,48,Male,Low,Regular,Average,2.0,6679610,48029,1977


## B_order - Change Datatype

In [73]:
B_order.head(3)

,Transaction_ID,Date,Ship Date,Delivery Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Shipping_Method,Payment_Method,Order_Status,Product ID,Customer_ID
0,8691788.0,2023-09-18,2023-09-22,2023-09-24,2023.0,September,22:03:55,3.0,108.028757,324.086270,Same-Day,Debit Card,Shipped,69417,37249.0
1,2174773.0,2023-12-31,2024-01-03,2024-01-04,2023.0,December,8:42:04,2.0,403.353907,806.707815,Standard,Credit Card,Processing,24176,69749.0
2,6679610.0,2023-04-26,2023-04-30,2023-05-03,2023.0,April,4:06:29,3.0,354.477600,1063.432799,Same-Day,Credit Card,Processing,48029,30192.0


In [74]:
cols = ['Transaction_ID', 'Customer_ID', 'Year']  # Specify column names
B_order[cols] = B_order[cols].apply(lambda x: x.astype(int))

In [75]:
B_order[['Transaction_ID', 'Customer_ID', 'Year']].dtypes

Transaction_ID    int32
Customer_ID       int32
Year              int32
dtype: object

In [76]:
B_order.head(3)

,Transaction_ID,Date,Ship Date,Delivery Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Shipping_Method,Payment_Method,Order_Status,Product ID,Customer_ID
0,8691788,2023-09-18,2023-09-22,2023-09-24,2023,September,22:03:55,3.0,108.028757,324.086270,Same-Day,Debit Card,Shipped,69417,37249
1,2174773,2023-12-31,2024-01-03,2024-01-04,2023,December,8:42:04,2.0,403.353907,806.707815,Standard,Credit Card,Processing,24176,69749
2,6679610,2023-04-26,2023-04-30,2023-05-03,2023,April,4:06:29,3.0,354.477600,1063.432799,Same-Day,Credit Card,Processing,48029,30192


## B_product - Change Datatype

In [77]:
B_product.head(3)

,Product ID,products,Product_Category,Product_Brand,Product_Type,Customer_ID,Transaction_ID
0,69417,Cycling shorts,Clothing,Nike,Shorts,37249.0,8691788.0
1,24176,Lenovo Tab,Electronics,Samsung,Tablet,69749.0,2174773.0
2,48029,Sports equipment,Books,Penguin Books,Children's,30192.0,6679610.0


In [78]:
cols = ['Transaction_ID', 'Customer_ID']  # Specify column names
B_product[cols] = B_product[cols].apply(lambda x: x.astype(int))

In [79]:
B_product[['Transaction_ID', 'Customer_ID']].dtypes

Transaction_ID    int32
Customer_ID       int32
dtype: object

In [80]:
B_product.head(3)

,Product ID,products,Product_Category,Product_Brand,Product_Type,Customer_ID,Transaction_ID
0,69417,Cycling shorts,Clothing,Nike,Shorts,37249,8691788
1,24176,Lenovo Tab,Electronics,Samsung,Tablet,69749,2174773
2,48029,Sports equipment,Books,Penguin Books,Children's,30192,6679610


---------------------------------------------------------------------------------------------------------------------------

## Save Processing data to Silver Layer

In [81]:
B_retail_data.to_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Silver_Data/retail_data.csv")

In [82]:
B_customer.to_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Silver_Data/customer.csv")

In [83]:
B_order.to_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Silver_Data/order.csv")

In [84]:
B_product.to_csv("D:/RAHUL-SIR-MCA-PROJECT-Copy/Silver_Data/product.csv")

--------------------------------------------------------------------------------------------------------------------------